# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

# Set Information

In [13]:
 # regression_1 --> regression
# regression_2 --> forecasting
# classification
model_mode = "classification"

In [14]:
if model_mode == "regression_1":
    ms_name_X = 'regression_energy_cleanLevel0_trainX'
    ms_name_y = 'regression_energy_cleanLevel0_trainy'
    feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    feature_y_list = ['value']
    split_mode ="windows_split"
    data_y_flag = 'true' # 이미 만들어진 Y 데이터를 활용함
    
    model_name = 'Regression_API_TEST'
    model_method = 'LSTM_rg'
    model_purpose = 'Regression'
    
    
elif model_mode == "regression_2":
    ms_name_X = 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX'
    ms_name_y = 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy'
    feature_X_list = ['Temperature', 'out_temp','sin_hour']
    feature_y_list = ['Temperature']
    split_mode = 'step_split'
    data_y_flag = False # Y데이터는 없음, X 에서 Y 데이터를 도출함
    
    model_name = 'Forecasting_API_TEST'
    model_method = 'LSTM_rg'
    model_purpose = 'Regression'
    
    
elif model_mode == "classification":
    ms_name_X = 'classification_actionPattern_cleanLevel0_trainX'
    ms_name_y = 'classification_actionPattern_cleanLevel0_trainy'
    feature_X_list = ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
    feature_y_list = ['value']
    split_mode = 'windows_split'
    data_y_flag = 'true'
    
    model_name = 'Classification_API_TEST'
    model_method = 'LSTM_cf'
    model_purpose = 'Classification'
    

In [15]:
model_mode

'classification'

In [17]:
if model_mode == "regression_1" or model_mode == "regression_2":
    print("1")
    regression_model_info ={"LSTM_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                         "GRU_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                         "CNN_1D_rg":{"output_channels":64,"kernel_size":3,"stride":1,"dropout":0.1,"padding":0},
                         "LSTM_FCNs_rg":{"num_layers":2,"lstm_dropout":0.4,"fc_dropout":0.1},
                         "FC_rg":{"dropout":2,"bias":0.4}}
    model_parameter = regression_model_info[model_method]
    train_parameter = {"lr":1e-4,"weight_decay":1e-6,"n_epochs":10,"batch_size":16}

elif model_mode == "classification":
    print("2")
    classification_model_info ={"LSTM_cf":{"hidden_size":64,"num_layers":2,"num_classes":2,"dropout":0.1,"bidirectional":True},
                         "GRU_cf":{"hidden_size":64,"num_layers":2,"num_classes":2,"dropout":0.1,"bidirectional":True},
                         "CNN_1D_cf":{"output_channels":64,"kernel_size":3,"stride":1,"dropout":0.1,"padding":0,"num_classes":6},
                         "LSTM_FCNs_cf":{"num_layers":2,"lstm_dropout":0.4,"fc_dropout":0.1,"num_classes":6},
                         "FC_cf":{"dropout":0.1,"bias":True}}
    model_parameter = classification_model_info[model_method]
    train_parameter = {"lr":1e-4,"weight_decay":1e-6,"n_epochs":10,"batch_size":16}

2


# Set Parameter

In [18]:
params = {
    'bk_name_X' : 'integration',
    'ms_name_X' : ms_name_X,
    'feature_X_list' : feature_X_list,
    'bk_name_y' : 'integration',
    'ms_name_y' : ms_name_y,
    'feature_y_list' : feature_y_list,
    'data_y_flag' : data_y_flag,
    'data_clean_option' : False,
    'model_purpose' : model_purpose,
    'model_method' : model_method,
    'model_name' : model_name,
    'model_tags' : 'tagstest',
    'train_parameter' : train_parameter,
    'model_parameter' : model_parameter
}

if model_mode == "regression_2":
    params['future_step'] = 2
    params['past_step'] = 24

In [19]:
params['data_y_flag']

'true'

# Training

## Use Function in API

In [20]:
from KETIAppDataServer2.model import dbModel
dbModel.DB_init()
from KETIAppDataServer2.API.machine_learning.model_train import set_train_data, training

In [21]:
train_X_array, train_y_array, val_X_array, val_y_array, model_info_meta = set_train_data(params)
result = training(train_X_array, train_y_array, val_X_array, val_y_array, params, model_info_meta)

True
['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
Make New scaler File
['value']
Make New scaler File
window_size: 128 nan_limit_num: 64
(752768, 9) (5881, 128, 9)
(5881, 1) (5881, 1)
window_size: 128 nan_limit_num: 64
(188288, 9) (1471, 128, 9)
(1471, 1) (1471, 1)
./Models/LSTM_cf/Classification_API_TEST/classification_actionPattern_cleanLevel0_trainX/d531eec11664669cff1f6a3ad9639012/model.pkl
Start training model

Epoch 1/10
train Loss: 0.3952 Acc: 0.8440
val Loss: 0.1050 Acc: 0.9979

Epoch 10/10
train Loss: 0.0136 Acc: 0.9974
val Loss: 0.0341 Acc: 0.9904

Training complete in 0m 59s
Best val Acc: 1.000000
========== Data Save Success ==========
======== OK ========


In [22]:
result

{'trainDataInfo': {'bucket_name': 'integration',
  'collection_name': 'classification_actionPattern',
  'ms_name': 'classification_actionPattern_cleanLevel0_trainX',
  'ingestion_type': 'multiple_ms_by_time',
  'ingestion_param': {'ms_list_info': [['bio_action_sensors',
     'accelerationTrain']],
   'start_time': '1992-01-11',
   'end_time': '2012-02-27'},
  'processing_type': 'step_3',
  'process_param': {'refine_param': {'remove_duplication': {'flag': False},
    'static_frequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certain_error_to_NaN': {'flag': False},
    'uncertain_error_to_NaN': {'flag': False}},
   'imputation_param': {'flag': False}},
  'integration_param': {'integration_frequency': 675.0,
   'param': {},
   'method': 'meta',
   'integration_duration': 'common'},
  'clean_level': 0},
 'dataSplitMode': 'windows_split',
 'featureList': ['col_0',
  'col_1',
  'col_2',
  'col_3',
  'col_4',
  'col_5',
  'col_6',
  'col_7',
  'col_8'],
 'target': ['value'